# SIT215 Artificial and Computational Intelligence Final Project
## October 2020
## Roger Middenway

This project uses the Frozen Lake 8x8 environment from OpenAI Gym. The Frozen Lake environment involves an 8x8 grid which consists of ice, holes, and a goal. The agent must traverse the map from the upper left corner, to the goal in the lower right, without falling into a hole. The twist is that the ice is slippery, causing the agent to move in a random, but non-inverse direction to the one chosen. 

The stochastic nature of this environment proves a challenge for reinforcement learning. We'll attempt to build a model using the Q Learning time difference model, chosen because of the discrete nature of the environment. We'll also compare the performance of a SARSA model, a time difference model very similar to Q Learning.

In [9]:
import gym
import numpy as np
from IPython.display import clear_output
from time import sleep

In [10]:
# Create the frozen lake environment, check its visualisation
env = gym.make('FrozenLake8x8-v0')
env.render()


SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


As seen in the render above, the map consists of the following:
red square - agent
F - ice
H - hole
G - goal

Now we create a function which prints each step of a stored set of frames.

In [24]:
# Play back resulting path of previous cell
def print_frames(frames, limit=np.inf):
    
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.2)
        if i > limit:
              break
        

To show the behaviour of an agent making random decisions, we use the action_space.sample() function. Note that the agent's actual movement doesn't always correspond with the intended movement printed above the map.

In [12]:
# Run an episode, applying a random action at each step
frames = []

env.reset()

while True:
    count += 1
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
    })
    if done:
        print_frames(frames)
        break

  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

Timestep: 60
State: 19
Action: 3
Reward: 0.0


## Q Learning

In order to implement the Q Learning algorithm, a Q table must first be created, representing the value of each action at each possible state. This will be initialised as a 2D array of zeros.

In [13]:
# Initialise a q table, in a separate cell so it can be built upon later
q_table = np.zeros([env.observation_space.n, env.action_space.n])

The Q Learning model takes 3 hyperparameters:
alpha controls the rate at which new information is integrated into the model
gamma controls the weighting placed on future rewards as opposed to instant rewards
epsilon controls the tendency of the model to explore a random step rather than the optimal one

The values used here are based on tuning the model over several iterations.

The Q Learning model will now be trained over 100000 episodes At each step in each episode, the Q Learning algorithm will: 
1. Choose and perform an action based on the Q table values (or at random if the epsilon check triggers).
2. Apply the reward function
3. Calculate a new reward value for the current action and current state, and update the Q Table. The formula used is: new value = old value + alpha * (reward + gamme * max value of next state - old value)

In [15]:
%%time
import random
from IPython.display import clear_output

# Set hyperparameters
alpha = 0.2 # learning speed, size of effect new reward has on weight
gamma = 0.9 # importance given to future rewards
epsilon = 0.1 # exploration tendency

# Initialise analysis variables
all_epochs = []
all_penalties = []
reward_count = 0

# Run 100000 episodes for training
for i in range(1, 100001):
    # Initialise environment, epochs, penalties and reward
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    done = False

    
    # Until done returns positive, make a random move based on epsilon, or find the best move based
    # on the q table.
    while not done:
        if random.uniform(0,1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])
        
        next_state, reward, done, info = env.step(action)
        
        # Apply reward function
        if reward:
            reward_count += 1
        reward *= 10
        if done and not reward:
            reward -= 1
        reward -= 0.1
        
        # Update Q table based on alpha and gamma
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = old_value + alpha * (reward + gamma * next_max - old_value)
        q_table[state, action] = new_value
        
        # Note a penalty if the penalty is below a certain threshold (NB: Not used in this version)
        if reward <= -0.5:
            penalties += 1
        
        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")
print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 2min 18s, sys: 18.8 s, total: 2min 36s
Wall time: 2min 20s


Now, for comparison, the same thing is done for the SARSA algorithm. The core difference between SARSA and Q Learning is that, where Q Learning takes the maximum value from the next state's Q table, SARSA actually makes the next move and uses the value received. This means that the value SARSA uses for the next action is subject to epsilon triggering a random move, so it may not receive the highest possible value. This results in a more conservative convergence of the Q table.

In [16]:
%%time
# Create q table for SARSA, epochs and penalties
q_table_sarsa = np.zeros([env.observation_space.n, env.action_space.n])
all_epochs = []
all_penalties = []


for i in range(1, 100001):
    # Initialise environment, epochs, penalties and reward
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    done = False
    
    # Choose initial action
    if random.uniform(0,1) < epsilon:
        action = env.action_space.sample()
    else:
        action = np.argmax(q_table_sarsa[state])
    
    # Until done returns positive, make a random move based on epsilon, or find the best move based
    # on the q table.
    while not done:
            
        next_state, reward, done, info = env.step(action)
        
        if random.uniform(0,1) < epsilon:
            next_action = env.action_space.sample()
        else:
            next_action = np.argmax(q_table_sarsa[next_state])
            
         # Apply reward adjustments
        if reward:
            reward_count += 1
        reward *= 10
        if done and not reward:
            reward -= 1
        reward -= 0.1
        
        # Update Q table based on alpha and gamma
        old_value = q_table_sarsa[state, action]
        next_value = q_table_sarsa[next_state, next_action]
        
        new_value = old_value + alpha * (reward + gamma * next_value - old_value)
        q_table_sarsa[state, action] = new_value
        
        # Note a penalty if the penalty is below a certain threshold (NB: Not used in this version)
        if reward <= -0.5:
            penalties += 1
        
        state = next_state
        action = next_action
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")
print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 1min 41s, sys: 15.8 s, total: 1min 57s
Wall time: 1min 44s


In [27]:
# Run 100 episodes based on q table generated in previous cell



# Set which algorithm's results to use
#table = q_table_sarsa
#table = q_table

def test_model(episodes, table):
    # Run through set number of episodes, referring to the q table for the correct move
    # NB: values are no longer being updated, and no epsilon is used
    frames = []
    total_epochs, total_penalties = 0, 0
    success_count = 0
    for _ in range(episodes):
        state = env.reset()
        epochs, penalties, reward = 0, 0, 0

        done = False

        while not done:
            if epochs > 50:
                break
            action = np.argmax(table[state])
            state, reward, done, info = env.step(action)
            success_count += reward
            if done and reward == 0:
                reward -= 0.5
            frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward
        })
            if reward <= -0.5:
                penalties += 1

            epochs += 1

        total_penalties += penalties
        total_epochs += epochs

    print(f"Results after {episodes} episodes:")
    print(f"Average timesteps per episode: {total_epochs / episodes}")
    print(f"Average penalties per episode: {total_penalties / episodes}")
    print('Success count:', success_count)
    return frames
print('Q Learning model performance')
ql_frames = test_model(100, q_table)
print('SARSA model performance')
sars_frames = test_model(100, q_table_sarsa)

Q Learning model performance
Results after 100 episodes:
Average timesteps per episode: 40.96
Average penalties per episode: 0.46
Success count: 12.0
SARSA model performance
Results after 100 episodes:
Average timesteps per episode: 45.5
Average penalties per episode: 0.29
Success count: 6.0


From this data we see that both models perform roughly the same although Q Learning has a slight edge, on average using fewer time steps per episode and achieving more successful traversals, however, it also incurs more penalties on average.

It should also be noted that the success rate of these models is around 10%. This alarming out come can be attributed to the random nature of the environment and the layout of the map. The best path to the goal is a narrow corridor on the right hand side, which goes past two holes. If we play back the episodes of the trained models' traversals, its behaviour will show us how it still has trouble finding the goal.

In [25]:
# Play back episodes

print_frames(ql_frames, 100)
print_frames(sars_frames, 100)

  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

Timestep: 102
State: 15
Action: 2
Reward: 0.0


This simple visualisation shows us the dominant action (that with the highest value) for each state in the Q table. Both models show a tendency to avoid the lower right hand side of the board, which is to be expected given that the optimal route is via the left. However, notice that the optimal actions for the lower right hand squares, just above the goal, involve walking off the map. Why do they not try to move towards the goal? 

This is because the random nature of the moves, which can adjust the direction by plus or minus 90 degrees, means that the safest method to get past these holes and to the reach the goal, is to move in the only direction that won't result in falling into the hole. Whether the agent reaches the goal or not, then becomes a matter of chance. This explains the low success rate.

In [32]:
# Print visualisation of dominant actions for each state in q table

def print_dir_map(table):
    directions = ['<','V','>','A']
    direction_matrix = [directions[np.argmax(state)] for state in table]
    for i in range(len(table)):
        if q_table[i][0] == 0:
            direction_matrix[i] = 'O'
    for i in range (8):
        line = ''
        for j in range(8):
            line += direction_matrix[i * 8 + j]
        print(line)

print('Q LEARN')
print_dir_map(q_table)
print('-'*8)
print('SARSA')
print_dir_map(q_table_sarsa)

Q LEARN
>>A>>>><
AAAAA>V>
AA<O>V>V
AAAV<O>V
AAAO>V>V
>OOV><O>
<OV<O>O>
>V<OVVVO
--------
SARSA
>>>>>AV>
AAAA>V>>
AA<O>A>>
A<<V<OA>
AA<O>V>>
>OO>A<O>
<OV<O>OV
<V<O>VVO


# Conclusion
This exploration shows the slight differences between the Q Learning and SARSA reinforcement learning algorithms. It also shows the ways in which a RL model adapts to a stochastic environment, opting for safer choices rather than direct paths. This could perhaps be a result of the reward function, however, as to whether adjusting the reward function to create a less risk averse model would lead to better results, would require further testing.